<a href="https://colab.research.google.com/github/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION/blob/main/Algoritmos_isaac_ropero_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  # AG3 - Actividad Guiada 3
  ## Isaac Ropero Mendez
  ### https://github.com/isaacropero/03MIAR_ALGORITMOS_DE_OPTIMIZACION/


He añadido algunas cosas ya que me resultó interesante el algoritmo. Pero sigue manteniendo la esencia de lo que se hizo en clase.

In [ ]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.2 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


In [ ]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos

In [ ]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
nodos = list(problem.get_nodes())

#Aristas
aristas = list(problem.get_edges())

In [ ]:
#Probamos algunas funciones del objeto problem

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#Distancia entre nodos
problem.get_weight(0, 1)

15

In [ ]:
def crear_solucion(nodos):
  solucion = [nodos[0]]
  for _ in nodos[1:]:
    solucion = solucion + [random.choice(list(set(nodos) - set(solucion)))] # Asignar un nuevo nodo de los que no estan en la lista de la solucion
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)



In [ ]:
print(crear_solucion(nodos))

[0, 12, 20, 8, 35, 30, 38, 16, 39, 4, 40, 18, 2, 9, 29, 7, 24, 14, 23, 37, 22, 32, 25, 31, 10, 5, 36, 11, 41, 21, 17, 34, 33, 3, 28, 19, 27, 15, 26, 6, 13, 1]


#BUSQUEDA ALEATORIA

In [ ]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 5000 iteraciones
solucion = busqueda_aleatoria(problem, 10000)

Mejor solución: [0, 10, 5, 3, 41, 24, 32, 18, 21, 36, 35, 28, 17, 37, 40, 22, 9, 38, 34, 31, 7, 15, 33, 20, 1, 2, 11, 29, 25, 8, 23, 30, 39, 27, 4, 13, 14, 16, 12, 19, 6, 26]
Distancia     : 3730


#BUSQUEDA LOCAL

## Vecindarios
2-opt -> genera_vecina_2opt </br>
3-opt -> genera_vecina_3opt

In [ ]:
def genera_vecina_2opt(solucion):
  mejor_solucion = []
  mejor_distancia = float('inf')

  for i in range(1, len(solucion)-1): # Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt. La posicion 0 no se puede mover.
    for j in range(i+1, len(solucion)):
      solucion[i], solucion[j] = solucion[j], solucion[i] # Hacer el swap

      distancia_vecina = distancia_total(solucion, problem)

      if distancia_vecina < mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = solucion[:]

      solucion[i], solucion[j] = solucion[j], solucion[i] # Revertir el swap

  return mejor_solucion

def genera_vecina_3opt(solucion):
  mejor_solucion = []
  mejor_distancia = float('inf')

  for i in range(1, len(solucion)-2):
    for j in range(i+1, len(solucion)-1):
      for k in range(j+1, len(solucion)):
        vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[k]] + solucion[j+1:k] + [solucion[i]] + solucion[k+1:]
        distancia_vecina = distancia_total(vecina, problem)

        if distancia_vecina <= mejor_distancia:
          mejor_distancia = distancia_vecina
          mejor_solucion = vecina

  return mejor_solucion

In [ ]:
solucion = crear_solucion(nodos)
coste = distancia_total(solucion, problem)
print(f'Solucion random sin aplicar nada: {coste}')
print(f'{solucion}\n')

solucion = busqueda_aleatoria(problem, 10000)
print("Solucion random buscando la de menor distancia -> busqueda_aleatoria():" , distancia_total(solucion, problem), "\n")

nueva_solucion = genera_vecina_2opt(solucion)
print("Distancia Mejor Solucion Local -> genera_vecina_2opt():", distancia_total(nueva_solucion, problem), "\n")

nueva_solucion = genera_vecina_3opt(solucion)
print("Distancia Mejor Solucion Local -> genera_vecina_3opt():", distancia_total(nueva_solucion, problem))

Solucion random sin aplicar nada: 5039
[0, 10, 18, 13, 26, 7, 23, 30, 19, 11, 33, 3, 22, 39, 6, 14, 21, 15, 36, 2, 27, 31, 32, 41, 9, 5, 37, 38, 16, 25, 34, 29, 1, 8, 20, 17, 28, 35, 24, 12, 40, 4]

Mejor solución: [0, 41, 10, 12, 7, 14, 5, 13, 37, 16, 34, 29, 38, 39, 22, 30, 8, 3, 36, 35, 24, 2, 27, 26, 15, 4, 31, 40, 21, 18, 25, 19, 28, 1, 6, 11, 33, 9, 23, 32, 20, 17]
Distancia     : 3771
Solucion random buscando la de menor distancia -> busqueda_aleatoria(): 3771 

Distancia Mejor Solucion Local -> genera_vecina_2opt(): 3344 

Distancia Mejor Solucion Local -> genera_vecina_3opt(): 3318


## Variable Neighborhood Search (VNS)

In [114]:
# Crear soluciones
solucion = busqueda_aleatoria(problem, 10000)
coste = distancia_total(solucion, problem)

mejor_solucion = solucion
mejor_coste = coste

vecindarios = [genera_vecina_2opt, genera_vecina_3opt]
print(f'{mejor_solucion}\n{mejor_coste}')

iteraciones = 0
max_iteraciones_permitidas = 10
while(iteraciones < max_iteraciones_permitidas):
  for vecindario in vecindarios:
    solucion = vecindario(mejor_solucion)

    coste = distancia_total(solucion, problem)

    if coste < mejor_coste:
      mejor_coste = coste
      mejor_solucion = solucion
      iteraciones = 0
    else:
      iteraciones += 1

print(f'{mejor_solucion}\n{mejor_coste}')

Mejor solución: [0, 4, 41, 40, 8, 28, 27, 23, 29, 10, 31, 33, 34, 20, 35, 36, 19, 6, 18, 5, 14, 9, 24, 11, 21, 38, 2, 7, 30, 15, 16, 32, 22, 17, 39, 25, 13, 37, 26, 3, 1, 12]
Distancia     : 3736
[0, 4, 41, 40, 8, 28, 27, 23, 29, 10, 31, 33, 34, 20, 35, 36, 19, 6, 18, 5, 14, 9, 24, 11, 21, 38, 2, 7, 30, 15, 16, 32, 22, 17, 39, 25, 13, 37, 26, 3, 1, 12]
3736
[0, 29, 9, 23, 41, 10, 25, 11, 12, 18, 32, 34, 33, 20, 35, 36, 16, 14, 19, 13, 8, 40, 24, 21, 39, 22, 38, 31, 17, 37, 15, 7, 30, 28, 27, 2, 3, 4, 26, 5, 6, 1]
1763
